In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse


### Step 1
Using the website [https://www.offiziellecharts.de/](https://www.offiziellecharts.de/) and web scraping with the help of BeautifulSoup, I saved the top 100 songs for each year from 2015 to 2022.


In [5]:
def scrape_year(year):
    """
    This function requests the official charts page for a given year,
    parses the HTML using Beautiful Soup, and returns a DataFrame that
    contains track positions, artist names, and track titles.
    """
    url = f"https://www.offiziellecharts.de/charts/single-jahr/for-date-{year}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error loading the page for {year}: HTTP {response.status_code}")
        return pd.DataFrame()
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all the table rows that contain track information.
    # In this case, it is assumed that the data is within <tr class="drill-down-link">
    rows = soup.find_all("tr", class_="drill-down-link")
    
    data = []
    for row in rows:
        # Extract the track position from <span class="this-week">
        pos_tag = row.find("span", class_="this-week")
        pos = pos_tag.get_text(strip=True) if pos_tag else None

        # Extract the artist name from <span class="info-artist">
        artist_tag = row.find("span", class_="info-artist")
        artist = artist_tag.get_text(strip=True) if artist_tag else None

        # Extract the track title from <span class="info-title">
        title_tag = row.find("span", class_="info-title")
        title = title_tag.get_text(strip=True) if title_tag else None
        
        data.append({"Position": pos, "Artist": artist, "Title": title})
    
    df = pd.DataFrame(data)
    return df

# Dictionary to store DataFrames for each year
yearly_dataframes = {}

# Process the years from 2015 to 2022 (inclusive)
for year in range(2015, 2023):
    print(f"Processing data for {year}...")
    df_year = scrape_year(year)
    yearly_dataframes[year] = df_year
    # Optionally, save the DataFrame to a separate CSV file
    df_year.to_csv(f"charts_{year}.csv", index=False, encoding="utf-8")
    print(f"Data for {year} saved (records found: {len(df_year)}).")



Processing data for 2015...
Data for 2015 saved (records found: 100).
Processing data for 2016...
Data for 2016 saved (records found: 100).
Processing data for 2017...
Data for 2017 saved (records found: 100).
Processing data for 2018...
Data for 2018 saved (records found: 100).
Processing data for 2019...
Data for 2019 saved (records found: 100).
Processing data for 2020...
Data for 2020 saved (records found: 100).
Processing data for 2021...
Data for 2021 saved (records found: 100).
Processing data for 2022...
Data for 2022 saved (records found: 100).


In [6]:
yearly_dataframes[2015]

,Position,Artist,Title
0,1,Omi,Cheerleader (Felix Jaehn Remix)
1,2,Lost Frequencies,Are You With Me
2,3,Felix Jaehn feat. Jasmine Thompson,Ain't Nobody (Loves Me Better)
3,4,Ellie Goulding,Love Me Like You Do
4,5,Major Lazer & DJ Snake feat. MØ,Lean On
...,...,...,...
95,96,Pitbull & Ne-Yo,Time Of Our Lives
96,97,K.I.Z. feat. Henning May,Hurra die Welt geht unter
97,98,Justin Bieber,Love Yourself
98,99,Mark Forster feat. Sido,Au revoir


In [7]:
yearly_dataframes[2022]

,Position,Artist,Title
0,1,DJ Robin & Schürze,Layla
1,2,Glass Animals,Heat Waves
2,3,Luciano,Beautiful Girl
3,4,Gayle,abcdefu
4,5,Ed Sheeran,Shivers
...,...,...,...
95,96,Bonez MC & RAF Camora,Letztes Mal
96,97,Bonez MC / RAF Camora,Sommer
97,98,Chris Rea,Driving Home For Christmas
98,99,Alle Farben feat. Kiddo,Alright


### Step 2
Using the song title and artist’s name, I searched for the MBID (MusicBrainz Identifier) on acousticbrainz.org.

In [ ]:
import requests
import time
import urllib.parse
import pandas as pd

def get_recording_mb(artist, title):
    """
    Search MusicBrainz for a recording using artist and title.
    Returns the MBID of the first result, or None if nothing is found.
    """
    query = f'artist:"{artist}" AND recording:"{title}"'
    url = f"https://musicbrainz.org/ws/2/recording/?query={urllib.parse.quote(query)}&fmt=json"
    headers = {"User-Agent": "YourAppName/1.0 (your_email@example.com)"}
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"MusicBrainz search error for '{artist} - {title}': HTTP {response.status_code}")
        return None

    data = response.json()
    recordings = data.get("recordings", [])
    if not recordings:
        print(f"No recording found for '{artist} - {title}'")
        return None

    return recordings[0].get("id")

mbid_yearly = {}

# --- Part 1: Collecting MBIDs for each track ---
for year, df in yearly_dataframes.items():
   
    results = []
    print(f"Processing MBIDs for year {year}...")
    for index, row in df.iterrows():
        artist = row['Artist']
        title = row['Title']
        print(f"Processing track: {artist} - {title}")
        
        # Retrieve MBID via MusicBrainz
        mbid = get_recording_mb(artist, title)
        time.sleep(1)  # Delay to respect MusicBrainz rate limits
        
        # Add MBID to the row data
        new_row = row.to_dict()
        new_row['MBID'] = mbid
        results.append(new_row)
    
    # Create a new DataFrame for the given year with the added MBID column
    new_df = pd.DataFrame(results)
    mbid_yearly[year] = new_df
    
    # Print a summary of successfully retrieved MBIDs
    found_count = new_df['MBID'].notna().sum()
    total_count = len(new_df)
    print(f"Finished processing {year}. Found MBID for {found_count} out of {total_count} tracks.\n")


Processing MBIDs for year 2015...
Processing track: Omi - Cheerleader (Felix Jaehn Remix)
Processing track: Lost Frequencies - Are You With Me
Processing track: Felix Jaehn feat. Jasmine Thompson - Ain't Nobody (Loves Me Better)
Processing track: Ellie Goulding - Love Me Like You Do
Processing track: Major Lazer & DJ Snake feat. MØ - Lean On
Processing track: Sido feat. Andreas Bourani - Astronaut
Processing track: Adele - Hello
Processing track: Robin Schulz feat. Francesco Yates - Sugar
Processing track: Wiz Khalifa feat. Charlie Puth - See You Again
Processing track: Kygo feat. Conrad - Firestone
Processing track: Philipp Dittberner & Marv - Wolke 4
Processing track: Namika - Lieblingsmensch
Processing track: Sarah Connor - Wie schön du bist
Processing track: Lost Frequencies feat. Janieck Devy - Reality
Processing track: Anna Naklab feat. Alle Farben & Younotus - Supergirl
Processing track: Jason Derulo - Want To Want Me
Processing track: Gestört aber GeiL & Koby Funk feat. Wincent

In [29]:
# Save  MBID DataFrames to a CSV file
for year, df in mbid_yearly.items():
    filename = f"tracks_mbids_{year}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"CSV file for {year} saved as '{filename}'")


#mbid_yearly[2015].to_csv('tracks_mbids_2015.csv', index=False, encoding='utf-8')
#print("CSV file for 2015 has been saved as 'tracks_mbids_2015.csv'")


CSV file for 2015 saved as 'tracks_mbids_2015.csv'
CSV file for 2016 saved as 'tracks_mbids_2016.csv'
CSV file for 2017 saved as 'tracks_mbids_2017.csv'
CSV file for 2018 saved as 'tracks_mbids_2018.csv'
CSV file for 2019 saved as 'tracks_mbids_2019.csv'
CSV file for 2020 saved as 'tracks_mbids_2020.csv'
CSV file for 2021 saved as 'tracks_mbids_2021.csv'
CSV file for 2022 saved as 'tracks_mbids_2022.csv'


In [11]:
mbid_yearly[2015]

,Position,Artist,Title,MBID
0,1,Omi,Cheerleader (Felix Jaehn Remix),38629b7b-d103-4659-a8ad-5b5634a59dbb
1,2,Lost Frequencies,Are You With Me,ee3ce507-2d82-4493-bebd-b55e9e8cb863
2,3,Felix Jaehn feat. Jasmine Thompson,Ain't Nobody (Loves Me Better),d61e5ae9-f9c0-4a5b-9f98-b0db7f98e439
3,4,Ellie Goulding,Love Me Like You Do,8f8a3f12-bf98-4599-8d3e-3e41b3a2a7e2
4,5,Major Lazer & DJ Snake feat. MØ,Lean On,bddbcf90-0aed-47bc-bf32-6ab21d7881de
...,...,...,...,...
95,96,Pitbull & Ne-Yo,Time Of Our Lives,e0dfb145-a0af-4df4-9f5c-89ad509106f3
96,97,K.I.Z. feat. Henning May,Hurra die Welt geht unter,b840777b-b91d-47d0-af7c-f5372cdd79eb
97,98,Justin Bieber,Love Yourself,a9adb57f-6812-4bd8-98e6-deadb328ebd3
98,99,Mark Forster feat. Sido,Au revoir,e8310a5d-203d-4d64-a9e0-1bb5f436635b


In [21]:
def search_high_audio_features(mbid):
     """
     Retrieve high-level acoustic features from AcousticBrainz using the provided MBID.
     Returns a dictionary with audio features if successful, or None otherwise.
     """
     url = f"https://acousticbrainz.org/api/v1/{mbid}/high-level"
     response = requests.get(url)
     if response.status_code != 200:
         print(f"AcousticBrainz error for MBID high-level {mbid}: HTTP {response.status_code}")
         return None
     return response.json()

def search_audio_features(mbid):
     """
     Retrieve low-level acoustic features from AcousticBrainz using the provided MBID.
     Returns a dictionary with audio features if successful, or None otherwise.
     """
     url = f"https://acousticbrainz.org/api/v1/{mbid}/low-level"
     response = requests.get(url)
     if response.status_code != 200:
         print(f"AcousticBrainz error for MBID low-level {mbid}: HTTP {response.status_code}")
         return None
     return response.json()

df_2015 = mbid_yearly[2015].copy()

# Prepare a list to collect the audio features for each track.
low_audio_features_list = []
high_audio_features_list = []

print("Iterating over all tracks in the 2015 dataset to retrieve audio features...")
for idx, row in df_2015.iterrows():
    mbid = row.get("MBID")
    if mbid:
        features_low = search_audio_features(mbid)
        features_high = search_high_audio_features(mbid)
        if (features_low is not None):
            print(f"Got low audio features for track '{row['Title']}' by {row['Artist']}.")
        else:
            print(f"Low audio features not found for track '{row['Title']}' by {row['Artist']}.")
        if (features_high is not None):
            print(f"Got high audio features for track '{row['Title']}' by {row['Artist']}.")
        else:
            print(f"High audio features not found for track '{row['Title']}' by {row['Artist']}.")
    else:
        print(f"No MBID for track '{row['Title']}' by {row['Artist']}")
        features = None
    low_audio_features_list.append(features_low)
    high_audio_features_list.append(features_high)
    # Delay to avoid hitting API rate limits
    time.sleep(1)



Iterating over all tracks in the 2015 dataset to retrieve audio features...
AcousticBrainz error for MBID low-level d15b0f43-ac17-4c01-95f1-687c5456093a: HTTP 404
AcousticBrainz error for MBID high-level d15b0f43-ac17-4c01-95f1-687c5456093a: HTTP 404
Low audio features not found for track 'Cheerleader (Felix Jaehn Remix)' by Omi.
High audio features not found for track 'Cheerleader (Felix Jaehn Remix)' by Omi.
Got low audio features for track 'Are You With Me' by Lost Frequencies.
Got high audio features for track 'Are You With Me' by Lost Frequencies.
AcousticBrainz error for MBID low-level bad77138-96a6-4b39-be33-2e5b0dd2a838: HTTP 404
AcousticBrainz error for MBID high-level bad77138-96a6-4b39-be33-2e5b0dd2a838: HTTP 404
Low audio features not found for track 'Ain't Nobody (Loves Me Better)' by Felix Jaehn feat. Jasmine Thompson.
High audio features not found for track 'Ain't Nobody (Loves Me Better)' by Felix Jaehn feat. Jasmine Thompson.
AcousticBrainz error for MBID low-level 161

In [22]:
# Add the audio features results to a new column in the DataFrame
df_2015["AudioFeatures_low"] = low_audio_features_list
df_2015["AudioFeatures_high"] = high_audio_features_list

df_2015.head()

,Position,Artist,Title,MBID,AudioFeatures_low,AudioFeatures_high
0,1,Omi,Cheerleader (Felix Jaehn Remix),d15b0f43-ac17-4c01-95f1-687c5456093a,None,None
1,2,Lost Frequencies,Are You With Me,a88cea49-b308-49a2-abd3-421c2aef0c80,{'lowlevel': {'average_loudness': 0.7344310879...,{'highlevel': {'danceability': {'all': {'dance...
2,3,Felix Jaehn feat. Jasmine Thompson,Ain't Nobody (Loves Me Better),bad77138-96a6-4b39-be33-2e5b0dd2a838,None,None
3,4,Ellie Goulding,Love Me Like You Do,161efdaf-9742-4e82-97df-7f0c4e59cbc3,None,None
4,5,Major Lazer & DJ Snake feat. MØ,Lean On,1e3a21a1-86a8-4cef-82be-b989c20ab113,None,None


In [25]:
low_found_count = df_2015['AudioFeatures_low'].notna().sum()
high_found_count = df_2015['AudioFeatures_high'].notna().sum()
total_count = len(df_2015)
print(f"Finished processing {year}. Found low features for {low_found_count} and high for {high_found_count} out of {total_count} tracks.\n")

Finished processing 2016. Found low features for 30 and high for 30 out of 100 tracks.



In [26]:
df_2015.to_csv("tracks_audio_features_2015.csv", index=False, encoding="utf-8")
print("Data with audio features saved as 'tracks_audio_features_2015.csv'.")

Data with audio features saved as 'tracks_audio_features_2015.csv'.


### Step 3
After that, using the MBID, I retrieved the audio features for each track by using the API functions of the acousticbrainz.org website.


In [30]:
all_years_audio_features = []

for year in range(2016, 2023):
    df_year = mbid_yearly[year].copy() 
    low_audio_features_list = []     
    high_audio_features_list = []     
    
    print(f"Processing year {year}...")
    for idx, row in df_year.iterrows():
        mbid = row.get("MBID")
        if mbid:
            features_low = search_audio_features(mbid)
            features_high = search_high_audio_features(mbid)
            if features_low is not None:
                print(f"Got low audio features for track '{row['Title']}' by {row['Artist']} in {year}.")
            else:
                print(f"Low audio features not found for track '{row['Title']}' by {row['Artist']} in {year}.")
            if features_high is not None:
                print(f"Got high audio features for track '{row['Title']}' by {row['Artist']} in {year}.")
            else:
                print(f"High audio features not found for track '{row['Title']}' by {row['Artist']} in {year}.")
        else:
            print(f"No MBID for track '{row['Title']}' by {row['Artist']} in {year}.")
            features_low = None
            features_high = None
        
        low_audio_features_list.append(features_low)
        high_audio_features_list.append(features_high)
      
        time.sleep(1)
        
    df_year["Low_AudioFeatures"] = low_audio_features_list
    df_year["High_AudioFeatures"] = high_audio_features_list
    df_year["Year"] = year
    

    all_years_audio_features.append(df_year)


final_df = pd.concat(all_years_audio_features, ignore_index=True)

final_df.to_csv("tracks_audio_features_2016_2022.csv", index=False, encoding="utf-8")
print("Combined results saved to 'tracks_audio_features_2016_2022.csv'.")

Processing year 2016...
Got low audio features for track 'Faded' by Alan Walker in 2016.
Got high audio features for track 'Faded' by Alan Walker in 2016.
AcousticBrainz error for MBID low-level 40eda342-9655-45f9-a620-97db83ea8bad: HTTP 404
AcousticBrainz error for MBID high-level 40eda342-9655-45f9-a620-97db83ea8bad: HTTP 404
Low audio features not found for track 'Die immer lacht' by Stereoact feat. Kerstin Ott in 2016.
High audio features not found for track 'Die immer lacht' by Stereoact feat. Kerstin Ott in 2016.
AcousticBrainz error for MBID low-level 7a6e19a3-41a2-46d9-8d25-9d7cb5c2363c: HTTP 404
AcousticBrainz error for MBID high-level 7a6e19a3-41a2-46d9-8d25-9d7cb5c2363c: HTTP 404
Low audio features not found for track 'Cheap Thrills' by Sia feat. Sean Paul in 2016.
High audio features not found for track 'Cheap Thrills' by Sia feat. Sean Paul in 2016.
AcousticBrainz error for MBID low-level f8564d8c-eb2a-48be-83ab-618fc44a647d: HTTP 404
AcousticBrainz error for MBID high-lev